In [1]:
import pdfplumber
import numpy as np
import pandas as pd


def extract_all_tables_from_pdf(pdf_file):
    all_tables = []
    with pdfplumber.open(pdf_file) as pdf:
        for page in pdf.pages:
            tables = page.extract_tables()
            for table in tables:
                # all_tables.append(pd.DataFrame(table[1:], columns=table[0]))
                all_tables.append(np.array(table))
    return all_tables


pdf_file = "GSTR3B.pdf"
all_tables = extract_all_tables_from_pdf(pdf_file)

In [2]:
t0=all_tables[0].astype(object).copy()
t0=np.array([t0[0][1:3], t0[1][1:3]])
t0

array([['Year', ''],
       ['Month', '']], dtype=object)

In [3]:
t1=all_tables[1].copy()
t1[1][0]='2.'
t1=np.array([t1[0][:4],t1[1][:4]])
t1

array([['1.', '', 'GSTIN', ''],
       ['2.', '', 'Legal name of the registered person',
        'Auto Populated']], dtype=object)

In [4]:
t2=all_tables[2].astype(object).copy()
t2[0][1]+=' value'
t2[0][2]+=' Tax'
t2[0][3]+=' Tax'
t2=np.delete(t2, [1,2], axis=0)
t2

array([['Nature of Supplies', 'Total Taxable value', 'Integrated Tax',
        'Central Tax', 'State/UT Tax', 'Cess'],
       ['1', '2', '3', '4', '5', '6'],
       ['(a) Outward taxable supplies (other than zero rated,\nnil rated and exempted)',
        '', '', '', '', ''],
       ['(b) Outward taxable supplies (zero rated )', '', '', '', '', ''],
       ['(c) Other outward supplies (Nil rated, exempted)', '', '', '',
        '', ''],
       ['(d) Inward supplies (liable to reverse charge)', '', '', '', '',
        ''],
       ['(e) Non-GST outward supplies', '', '', '', '', '']], dtype=object)

In [5]:
t3=all_tables[3].astype(object).copy()

res=np.concatenate((t3[0,2:5],t3[0,6:])).copy()
res=np.vstack((res, [i for i in range(1,5)]))
res[0][0]=''
res[0][1]+=' (State/UT)'

for i in t3[3:,0]:
    res=np.vstack((res, [i, '', '', '']))
t3=res
t3

array([['', 'Place of Supply (State/UT)', 'Total Taxable value',
        'Amount of Integrated Tax'],
       [1, 2, 3, 4],
       ['Supplies made to\nUnregistered\nPersons', '', '', ''],
       ['Supplies made to Composition\nTaxable Persons', '', '', ''],
       ['Supplies made to UIN holders', '', '', '']], dtype=object)

In [6]:
t4=all_tables[4].astype(object).copy()
t4=np.delete(t4, [1], axis=0)
t4=t4
t4

array([['Details', 'Integrated Tax', 'Central Tax', 'State/UT Tax',
        'Cess'],
       ['1', '2', '3', '4', '5'],
       ['(A) ITC Available (whether in full or part)', '', '', '', ''],
       ['(1) Import of goods', '', '', '', ''],
       ['(2) Import of services', '', '', '', ''],
       ['(3) Inward supplies liable to reverse charge (other\nthan 1 & 2 above)',
        '', '', '', ''],
       ['(4) Inward supplies from ISD', '', '', '', ''],
       ['(5) All other ITC', '', '', '', ''],
       ['(B) ITC Reversed', '', '', '', ''],
       ['(1) As per rules 42 & 43 of CGST Rules', '', '', '', ''],
       ['(2) Others', '', '', '', ''],
       ['(C) Net ITC Available (A) – (B)', '', '', '', ''],
       ['(D) Ineligible ITC', '', '', '', ''],
       ['(1) As per section 17(5)', '', '', '', ''],
       ['(2) Others', '', '', '', '']], dtype=object)

In [7]:
t5=all_tables[5].astype(object).copy()
t5

array([['Nature of supplies', 'Inter-State supplies',
        'Intra-State supplies'],
       ['1', '2', '3'],
       ['From a supplier under composition scheme, Exempt and Nil rated\nsupply',
        '', ''],
       ['Non GST supply', '', '']], dtype=object)

In [8]:
t6=all_tables[6].astype(object).copy()
t6

array([['Description', 'Tax\npayable', 'Paid through ITC', None, None,
        None, 'Tax paid\nTDS./TCS', 'Tax/Cess\npaid in\ncash',
        'Interest', 'Late\nFee'],
       [None, None, 'Integrated\nTax', 'Central\nTax', 'State/UT\nTax',
        'Cess', None, None, None, None],
       ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'],
       ['Integrated Tax', '', '', '', '', '', '', '', '', ''],
       ['Central Tax', '', '', '', '', '', '', '', '', ''],
       ['State/UT Tax', '', '', '', '', '', '', '', '', ''],
       ['Cess', '', '', '', '', '', '', '', '', '']], dtype=object)

In [9]:
t7=all_tables[7].astype(object).copy()
t7

array([['Details', 'Integrated Tax', 'Central Tax', 'State/UT Tax'],
       ['1', '2', '3', '4'],
       ['TDS', '', '', ''],
       ['TCS', '', '', '']], dtype=object)

In [18]:
l=[pd.DataFrame(i[1:], columns=i[0]) for i in [t0,t1,t2,t3,t4,t5,t6,t7]]

In [19]:
l

[    Year  
 0  Month  ,
    1.                                  GSTIN                
 0  2.    Legal name of the registered person  Auto Populated,
                                   Nature of Supplies Total Taxable value  \
 0                                                  1                   2   
 1  (a) Outward taxable supplies (other than zero ...                       
 2         (b) Outward taxable supplies (zero rated )                       
 3   (c) Other outward supplies (Nil rated, exempted)                       
 4     (d) Inward supplies (liable to reverse charge)                       
 5                       (e) Non-GST outward supplies                       
 
   Integrated Tax Central Tax State/UT Tax Cess  
 0              3           4            5    6  
 1                                               
 2                                               
 3                                               
 4                                               
 5       

In [27]:
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Spacer

def merge_tables_to_pdf(all_tables, output_pdf):
    doc = SimpleDocTemplate(output_pdf, pagesize=letter)
    elements = []
    for idx, table in enumerate(all_tables):
        data = [table.columns.tolist()] + table.values.tolist()
        t = Table(data)
        t.setStyle(TableStyle([('BACKGROUND', (0, 0), (-1, 0), colors.lightblue),
                               ('TEXTCOLOR', (0, 0), (-1, 0), colors.black),
                               ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                               ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                               ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
                               ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
                               ('GRID', (0, 0), (-1, -1), 1, colors.black)]))
        elements.append(t)
        # Add space between tables except for the last one
        if idx < len(all_tables) - 1:
            elements.append(Spacer(1, 20))  # Add 20 units of space between tables
    doc.build(elements)

output_pdf = "merged_tables.pdf"
merge_tables_to_pdf(l, output_pdf)

In [31]:
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Spacer, PageBreak
from reportlab.pdfgen import canvas

def calculate_table_height(table, canvas, doc):
    """
    Calculate the height of the table by rendering it onto a canvas and measuring its height.
    """
    table.wrap(doc.width, doc.height)
    table.drawOn(canvas, 0, 0)
    return table._height

def merge_tables_to_pdf(all_tables, output_pdf):
    doc = SimpleDocTemplate(output_pdf, pagesize=letter)
    elements = []
    for idx, table in enumerate(all_tables):
        data = [table.columns.tolist()] + table.values.tolist()
        t = Table(data)
        t.setStyle(TableStyle([('BACKGROUND', (0, 0), (-1, 0), colors.lightblue),
                               ('TEXTCOLOR', (0, 0), (-1, 0), colors.black),
                               ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                               ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                               ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
                               ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
                               ('GRID', (0, 0), (-1, -1), 1, colors.black)]))
        
        # Calculate the height of the table
        canvas = canvas.Canvas("temp.pdf")
        table_height = calculate_table_height(t, canvas, doc)
        
        # Add the table to elements
        elements.append(t)
        
        # Check if the table spans multiple pages
        remaining_height = doc.height - doc.bottomMargin - doc.topMargin
        if table_height > remaining_height:
            elements.append(PageBreak())  # Add a page break
        
    doc.build(elements)

output_pdf = "merged_tables.pdf"
merge_tables_to_pdf(l, output_pdf)


UnboundLocalError: cannot access local variable 'canvas' where it is not associated with a value